In [10]:
# Bibliotecas necessárias

import pandas as pd
import zipfile 
import requests, io, os

In [11]:
# Link com os dados que quero analisar referente aos filmes exibidos no Brasil

link_zip = 'https://dados.ancine.gov.br/dados-abertos/BilheteriaObrasDia.zip'

In [12]:
# Baixando o arquivo .zip e extraindo os arquivos .csv

pasta = '/Users/bentofh/Documents/GitHub/ancine/arquivos_csv'

getfolder = requests.get(link_zip)
unzipfolder = zipfile.ZipFile(io.BytesIO(getfolder.content))
unzipfolder.extractall(pasta)
os.chdir(pasta)
csv_files = [f for f in os.listdir() if f.endswith('.csv')]

In [35]:
# Transformando os arquivos em parquet

anos = range(2014, 2024)

for ano in anos:
    arquivo_csv = os.path.join(pasta, f'BilheteriaObrasDia_{ano}.csv')
    if os.path.isfile(arquivo_csv):
        try:
            df = pd.read_csv(arquivo_csv, encoding='utf8', delimiter=';')
        except UnicodeDecodeError:
            df = pd.read_csv(arquivo_csv, encoding='ISO-8859-1', delimiter=';')
    arquivo_parquet = os.path.join(pasta, f'BilheteriaObrasDia_{ano}.parquet')
    df.to_parquet(arquivo_parquet)

In [36]:
# Lista para armazenar os dataframes
dataframes = []

# Itera sobre os arquivos na pasta
for ano in range(2014, 2024):
    arquivo = os.path.join(pasta, f'BilheteriaObrasDia_{ano}.parquet')
    df = pd.read_parquet(arquivo)
    dataframes.append(df)

In [38]:
# Combina todos os dataframes em um único
df_total = pd.concat(dataframes, ignore_index=True)

# Filtra por cidade
df_londrina = df_total[df_total['MUNICIPIO_COMPLEXO'] == 'LONDRINA']

# Salva o dataframe filtrado em parquet
df_londrina.to_parquet('Bilheteria_Londrina.parquet', index=False)